##  Building on Dash: Interactions and Mapping

Following up on last week's introduction to Dash's core components, interactivity, and basic syntax, this session dives deeper into creating dynamic and informative dashboards.

We'll explore **Advanced Interactions**.  Go beyond the basics and discover how Dash components can work together to provide a richer user experience.

**Need a refresher on Dash fundamentals?** Refer to the following resources:

* **Layout:** [https://dash.plotly.com/layout](https://dash.plotly.com/layout)
* **Callbacks:** [https://dash.plotly.com/basic-callbacks](https://dash.plotly.com/basic-callbacks)
* **Interactive Graphing:** [https://dash.plotly.com/interactive-graphing](https://dash.plotly.com/interactive-graphing)
* **Sharing Data in Callbacks:** [https://dash.plotly.com/r/sharing-data-between-callbacks](https://dash.plotly.com/r/sharing-data-between-callbacks)

# Basic Dash Callbacks
This chapter focuses on implementing interactivity within Dash applications using callback functions. These functions are triggered automatically by Dash whenever the value (property) of an input component changes. Consequently, the function updates a designated property within another component (the output).

A basic example is provided to illustrate the functionality of callbacks in a simple interactive Dash app.

```python
from dash import Dash, dcc, html, Input, Output, callback

app = Dash(__name__)

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),

])

@callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)

def update_output_div(input_value):
    return f'Output: {input_value}'


if __name__ == '__main__':
    app.run(debug=True)
```

In [1]:
#install dash library, if you haven't yet
#!pip install dash

In [2]:
#run the example here

###Understanding Callbacks in Dash Applications

```python
@callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)
```

This section dissects a basic Dash app example to demonstrate callback functionalities.

* **Callback Decorator:**
    - The `@callback` decorator identifies the function and its role within the application.
    - Learn more about its usage through the provided reference. [(link to reference on using @callback decorator)]
* **Function Breakdown:**
    - **Functionality:** This function is automatically called by Dash whenever the user interacts with the input component (text box). The update reflects in the designated output component (HTML div) on the page.
    - **Function Naming:** While any name can be used, it's recommended to choose a name that reflects the function's output.
    - **Arguments:** Similar to regular Python functions, arguments can be named freely. However, their order within the function must match the order defined in the decorator. Optional named keyword arguments can also be used. Refer to the "Flexible Callback Signatures" chapter for more details.
    - **Component References:** IDs assigned to Dash components in the application layout must be used exactly when referencing them as inputs or outputs within the `@callback` decorator.
    - **Decorator Placement:** The `@callback` decorator needs to be directly placed above the function declaration. Any blank lines will cause the callback registration to fail.
    - **Decorator Syntax:** For those interested in the underlying mechanism, the provided references offer further explanation on decorators.
  - [Decorator Basics](https://stackoverflow.com/questions/739654/how-do-i-make-function-decorators-and-chain-them-together/1594484#1594484)
  - [PEP 318 – Decorators for Functions and Methods](https://peps.python.org/pep-0318/#current-syntax)

* **Inputs and Outputs in Dash:**
    - In Dash, application inputs and outputs are essentially component properties.
    - This example showcases the "value" property of the component with ID "my-input" as the input and the "children" property of the component with ID "my-output" as the output.

* **Callback Execution:**
    - Whenever an input property changes, the function associated with the `@callback` decorator is automatically triggered.
    - Dash supplies the function with the updated input value as an argument. Subsequently, Dash updates the output component's property based on the function's return value.

* **Optional Keywords:**
    - The `component_id` and `component_property` keywords are optional for both input and output objects (each requiring only two arguments). They are included here for clarity but omitted elsewhere for conciseness.

* **Distinguishing Objects:**
    - It's crucial to differentiate between `dash.dependencies.Input` (used for callback definitions) and `dcc.Input` (the actual component).

* **Initial State:**
    - The `children` property of the "my-output" component is intentionally left without a value in the layout.
    - Upon starting the app, Dash automatically invokes all callbacks with the initial values of input components, establishing the initial state of the output components.

* **Analogy:**
    - This behavior resembles Microsoft Excel, where changes in one cell (input) automatically update dependent cells (outputs). This concept is called "Reactive Programming" due to the automatic response of outputs to input modifications.

* **Dynamic Updates:**
    - Recall that every component is defined by its set of keyword arguments. These arguments become component properties, which callbacks can dynamically modify.
    - Commonly, updates involve modifying the `children` property of HTML components to display new text or the `figure` property of `dcc.Graph` components to present new data. Additionally, callbacks can update component styles or even the options available in a `dcc.Dropdown` component.


In [3]:
#run the example here again

**Similarities to Spreadsheets:**

- An analogy can be drawn between Dash applications and spreadsheet software like Microsoft Excel. In both cases:
    - A change in one element (acting as the input) triggers an automatic update in other elements that depend on it (the outputs).

- This behavior is referred to as **Reactive Programming**. It signifies that the outputs react automatically to modifications in the inputs.

**Component Properties and Callbacks:**

- As previously mentioned, each Dash component is defined by its set of keyword arguments.
- These arguments become properties associated with the component upon creation in Python.

- Leveraging Dash's interactivity features, callbacks enable dynamic updates to any of these properties.

**Common Use Cases for Callbacks:**

- Frequently, callbacks are used to modify:
    - The `children` property of HTML components: This allows for displaying new text content within the component.
    - The `figure` property of `dcc.Graph` components: This facilitates the visualization of updated data.

- Additionally, callbacks can be employed to dynamically adjust:
    - The style of a component.
    - The available options within a `dcc.Dropdown` component.

**Example: dcc.Slider and dcc.Graph Interaction:**

- The following section presents another example where a `dcc.Slider` component is used to update a `dcc.Graph` component. This further demonstrates the application of callbacks in creating interactive Dash applications.


## Dash App Layout With Figure and Slider

Use this code for reference
```python
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        df['year'].min(),
        df['year'].max(),
        step=None,
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        id='year-slider'
    )
])


@callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig


if __name__ == '__main__':
    app.run(debug=True)
```

In [4]:
#run the example here

**Input and Output Properties in this Example:**
```python
@callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))
```
- The input for this specific application is the "value" property associated with the `dcc.Slider` component.
- The "figure" property of the `dcc.Graph` component serves as the application's output.

In [5]:
#run the example here
#play around with the sliders

**Callback Execution:**

- Whenever a modification occurs in the "value" property of the `dcc.Slider`, Dash automatically triggers the callback function named `update_figure`.
- This function receives the updated value as an argument.

**Functionality of the callback function (update_figure):**
```python
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig
```
1. **Data Filtering:** The function utilizes the received value to filter the provided dataframe.
2. **Figure Creation:** Based on the filtered data, a figure object is constructed.
3. **Output Update:** The constructed figure object is returned to the Dash application, consequently updating the content displayed by the `dcc.Graph` component.


In [6]:
#run the example
#load only the data frame

**Data Handling and Callbacks:**

* **Data Loading:**
    - The `df = pd.read_csv('...')` statement within the app's initialization phase utilizes the Pandas library to load the dataframe.
    - This dataframe (`df`) is stored in the application's global state, making it accessible by callback functions.

* **Efficiency Considerations:**
    - Loading data into memory can be resource-intensive.
    - By loading and storing the data (`df`) during app initialization (outside of callback functions), this operation occurs only once: at server startup.
    - This ensures that subsequent user interactions or app visits utilize the pre-loaded data (`df`), which is already in memory.

* **Global vs. Local Data Management:**
    - Whenever possible, computationally expensive operations like data download or querying should be performed within the global scope of the app, rather than within individual callback functions.

```python
@callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig
```

* **Data Modification within Callbacks:**
    - It's crucial to note that callbacks **do not directly modify** the original data.
    - They solely create copies of the dataframe through filtering operations using Pandas functionalities.

* **Importance of Data Isolation:**
    - This emphasis on data manipulation within callbacks highlights a critical aspect: callbacks should never alter variables outside their designated scope.
    - Modifications within callbacks to global state variables can lead to unintended consequences:
        - One user's session might influence another user's experience.
        - When deployed on multiple processes or threads, these modifications won't be shared across sessions, potentially causing inconsistencies.

* **Visual Enhancements:**
    - The `layout.transition` property is set to `True` to enable transitions during data updates.
    - This creates a visual effect where the chart smoothly animates from one state (data representation) to the next, enhancing user experience.

## Dash App With Multiple Inputs

**Multiple Inputs for a Single Output in Dash:**

Dash applications allow a single output component to be influenced by multiple input components.

For discussion, refer to this example:
```python
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px

import pandas as pd

app = Dash(__name__)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='xaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='xaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='yaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},

    )
])


@callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')

    return fig


if __name__ == '__main__':
    app.run(debug=True)
```

In [7]:
#run the example here

**Illustrative Example:**

This example demonstrates how five input components can be linked to a single output component:

```python
@callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))
```

* **Input Components:**
    - Two `dcc.Dropdown` components (their "value" properties)
    - Two `dcc.RadioItems` components (their selected options)
    - One `dcc.Slider` component (its "value")
    > **Key Observation:** Within the `app.callback` decorator, note that all five `Input` definitions are listed after the single `Output` definition. This structure informs Dash about the relationship between these components.

* **Output Component:**
    - The "figure" property of a `dcc.Graph` component





In [8]:
#run the app here
# do at least 3 types of interactions leading to 3 distinct views of the visualization

**Callback Trigger and Input Handling in this Example:**

* **Callback Execution:**
    - This specific example configures the callback to trigger whenever the value property of any of the following components changes:
        - `dcc.Dropdown` components
        - `dcc.Slider` component
        - `dcc.RadioItems` components

* **Input Arguments:**
    - The callback function receives arguments that correspond to the current values of each specified "input" property.
    - The order of these arguments strictly follows the order in which the inputs were defined within the `app.callback` decorator.


In [9]:
#run the app here

"""
do at least 3 types of interactions
leading to 3 distinct views of the visualization
"""

'\ndo at least 3 types of interactions\nleading to 3 distinct views of the visualization\n'

* **State Acquisition:**
    - Even though a user can only modify the value of a single component at a time, Dash gathers the current state (value) of all specified input properties.
    - This comprehensive data is then passed as arguments to the callback function.

* **Guaranteed Updates:**
    - Callback functions are consistently provided with the most up-to-date application state, ensuring they operate on the latest information.

## Updating Multiple Outputs in Dash Callbacks

The examples presented thus far have demonstrated callbacks that modify a single output property. However, Dash offers the capability to update multiple outputs simultaneously.

For the discussion, refer to this example:
```python
from dash import Dash, dcc, html, Input, Output, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Input(
        id='num-multi',
        type='number',
        value=5
    ),
    html.Table([
        html.Tr([html.Td(['x', html.Sup(2)]), html.Td(id='square')]),
        html.Tr([html.Td(['x', html.Sup(3)]), html.Td(id='cube')]),
        html.Tr([html.Td([2, html.Sup('x')]), html.Td(id='twos')]),
        html.Tr([html.Td([3, html.Sup('x')]), html.Td(id='threes')]),
        html.Tr([html.Td(['x', html.Sup('x')]), html.Td(id='x^x')]),
    ]),
])


@callback(
    Output('square', 'children'),
    Output('cube', 'children'),
    Output('twos', 'children'),
    Output('threes', 'children'),
    Output('x^x', 'children'),
    Input('num-multi', 'value'))
def callback_a(x):
    return x**2, x**3, 2**x, 3**x, x**x


if __name__ == '__main__':
    app.run(debug=True)
````

In [10]:
#run the example here
# try the following inputs: 9,19,95

```python
@callback(
    Output('square', 'children'),
    Output('cube', 'children'),
    Output('twos', 'children'),
    Output('threes', 'children'),
    Output('x^x', 'children'),
    Input('num-multi', 'value'))
def callback_a(x):
    return x**2, x**3, 2**x, 3**x, x**x
```

**Mechanism:**

1. **Output Specification:** Within the `app.callback` decorator, list all the properties you intend to update.

2. **Callback Return Value:** The callback function should return the same number of elements as the number of specified outputs.

**Practical Application:**

This functionality is particularly beneficial when two or more outputs rely on the same computationally expensive intermediate result, such as a slow database query. By calculating the result once within the callback and utilizing it to update multiple outputs, you can enhance the application's efficiency.


In [11]:
# refactor the code such that:

"""
Instead of x^2,x^3,2*x,3*x,x*x

Produce a calculator which produces the following sequences:

2(x+3),5*sqrt(x),pi*(x^2))

additional notes:

import numpy as np
np.pi

sqrt = x**(1/2)

Send a screenshot that this is working to ria.flora@dlsu.edu.ph
this will serve as part of your in class activity for today

THIS IS AN INDIVIDUAL ACTIVITY
"""


'\nInstead of x^2,x^3,2*x,3*x,x*x\n\nProduce a calculator which produces the following sequences:\n\n2(x+3),5*sqrt(x),pi*(x^2))\n\nadditional notes:\n\nimport numpy as np\nnp.pi\n\nsqrt = x**(1/2)\n\nSend a screenshot that this is working to ria.flora@dlsu.edu.ph\nthis will serve as part of your in class activity for today\n\nTHIS IS AN INDIVIDUAL ACTIVITY\n'

## Dash App With Chained Callbacks
For discussion purposes, refer to this example:

```python
from dash import Dash, dcc, html, Input, Output, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

all_options = {
    'America': ['New York City', 'San Francisco', 'Cincinnati'],
    'Canada': ['Montréal', 'Toronto', 'Ottawa']
}
app.layout = html.Div([
    dcc.RadioItems(
        list(all_options.keys()),
        'America',
        id='countries-radio',
    ),

    html.Hr(),

    dcc.RadioItems(id='cities-radio'),

    html.Hr(),

    html.Div(id='display-selected-values')
])


@callback(
    Output('cities-radio', 'options'),
    Input('countries-radio', 'value'))
def set_cities_options(selected_country):
    return [{'label': i, 'value': i} for i in all_options[selected_country]]


@callback(
    Output('cities-radio', 'value'),
    Input('cities-radio', 'options'))
def set_cities_value(available_options):
    return available_options[0]['value']


@callback(
    Output('display-selected-values', 'children'),
    Input('countries-radio', 'value'),
    Input('cities-radio', 'value'))
def set_display_children(selected_country, selected_city):
    return f'{selected_city} is a city in {selected_country}'


if __name__ == '__main__':
    app.run(debug=True)
```

In [12]:
#run the example here

This example demonstrates a series of interconnected callbacks:

```python
@callback(
    Output('cities-radio', 'options'),
    Input('countries-radio', 'value'))
def set_cities_options(selected_country):
    return [{'label': i, 'value': i} for i in all_options[selected_country]]
```

**Updating Available Options**
  - The first callback dynamically updates the available options within the second `dcc.RadioItems` component.
  - This update is based on the user's selection in the first `dcc.RadioItems` component.

```python
@callback(
    Output('cities-radio', 'value'),
    Input('cities-radio', 'options'))
def set_cities_value(available_options):
    return available_options[0]['value']
```
**Setting Initial Value:**
  - The second callback establishes an initial value for the second `dcc.RadioItems` component.
  - This value is set to the first element within the updated options list whenever the `options` property changes.


```python
@callback(
    Output('display-selected-values', 'children'),
    Input('countries-radio', 'value'),
    Input('cities-radio', 'value'))
def set_display_children(selected_country, selected_city):
    return f'{selected_city} is a city in {selected_country}'
```
**Displaying Selected Values:**
  - The final callback is responsible for displaying the currently selected values of both components.


In [13]:
#run the example here again

**Ensuring Data Consistency:**

- It's noteworthy that when modifying the value in the "countries" `dcc.RadioItems` component, Dash implements a specific execution order.
- Dash waits for the update of the "cities" component's value before calling the final callback.
- This mechanism safeguards against the scenario where callbacks might be called with conflicting information, such as "America" and "Montréal" (which wouldn't be a valid combination in this context).

In essence, these chained callbacks work together to maintain a consistent state within the application and ensure the displayed information reflects the user's selections accurately.

In [14]:
# refactor the code

"""Refactor the code such that you are showing at least 3 cities in the Province of Cebu vs Province of Negros Occidental
Screenshot your refactored code and proof that it is working.
This, again, forms part of your in-class activity for today -- add it to the items instructed above"""

'Refactor the code such that you are showing at least 3 cities in the Province of Cebu vs Province of Negros Occidental\nScreenshot your refactored code and proof that it is working.\nThis, again, forms part of your in-class activity for today -- add it to the items instructed above'

## Dash App With State

For discussion on this section, refer to this example:

```python
from dash import Dash, dcc, html, Input, Output, callback

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Input(id="input-1", type="text", value="Montréal"),
    dcc.Input(id="input-2", type="text", value="Canada"),
    html.Div(id="number-output"),
])


@callback(
    Output("number-output", "children"),
    Input("input-1", "value"),
    Input("input-2", "value"),
)
def update_output(input1, input2):
    return f'Input 1 is "{input1}" and Input 2 is "{input2}"'


if __name__ == "__main__":
    app.run(debug=True)
```

**Handling Form-like Interactions in Dash:**

Dash applications can sometimes incorporate functionalities similar to forms. In such scenarios, you might want to:

* **Defer Input Reading:** Instead of capturing the value of an input component immediately after each change, you might prefer to wait until the user has finished entering all the required information.

* **Example:** This example demonstrates a callback that triggers whenever any of the specified input properties are modified.

**State for Capturing User Input:**

- Dash provides the `State` argument within the `app.callback` decorator.
- This functionality allows you to access the current values of specific components without triggering additional callbacks.
- In essence, `State` serves as a way to pass supplementary information to the callback function without causing immediate updates.



In [15]:
#run the example here
"""
- Try interacting with the provided example by entering data in the input fields.
- Observe how the callback is not triggered with every keystroke,
but rather waits for the user to potentially complete entering all the information."""

'\n- Try interacting with the provided example by entering data in the input fields.\n- Observe how the callback is not triggered with every keystroke,\nbut rather waits for the user to potentially complete entering all the information.'

Here's the same example as above but with the two `dcc.Input` components as State and a new button component as an `Input`.

```python
from dash import Dash, dcc, html, Input, Output, State, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Input(id='input-1-state', type='text', value='Montréal'),
    dcc.Input(id='input-2-state', type='text', value='Canada'),
    html.Button(id='submit-button-state', n_clicks=0, children='Submit'),
    html.Div(id='output-state')
])


@callback(Output('output-state', 'children'),
              Input('submit-button-state', 'n_clicks'),
              State('input-1-state', 'value'),
              State('input-2-state', 'value'))
def update_output(n_clicks, input1, input2):
    return f'''
        The Button has been pressed {n_clicks} times,
        Input 1 is "{input1}",
        and Input 2 is "{input2}"
    '''


if __name__ == '__main__':
    app.run(debug=True)
```



In [16]:
#run the example here

**User Interaction and Callback Execution:**

This example showcases a distinct approach to handling user input and callback execution:

* **Input Fields and State:**
    - The values entered in the two `dcc.Input` components are captured using the `State` arguments within the `app.callback` decorator.
    - This implies that changes within these input fields **do not directly trigger the callback**.

* **Button as Trigger:**
    - The callback is solely activated upon clicking the `dcc.Button` component.

* **State Value Persistence:**
    - Even though modifying the input fields doesn't directly trigger the callback, their current values are still accessible within the callback function.
    - These values are passed as arguments to the callback when it eventually executes.

* **n_clicks Property:**
    - The `n_clicks` property of the `html.Button` component is used to monitor button interactions.
    - This property increases by 1 every time the button is clicked.

* **Applicability of n_clicks:**
    - The `n_clicks` property is a general feature available in most components within Dash HTML Components (`dash.html`).
    - However, it finds its most practical application with buttons, as it effectively tracks button presses.

In essence, this approach decouples user input from immediate action. Users can enter data at their own pace, and the callback is only invoked when they explicitly confirm their input by clicking the button. This pattern is useful for scenarios where validation or processing of information should occur only after user confirmation.


## Passing Components Into Callbacks Instead of IDs

**Component Identification and Referencing in Callbacks:**

* Throughout previous examples, we've established a practice of assigning unique identifiers (IDs) to components during the application layout definition.
* These IDs play a crucial role in referencing the components within callback functions. They are specified as arguments for both inputs and outputs.

* **Illustrative Example:**
    - The first example showcases a `dcc.Input` component designated with the ID 'my-input' and an `html.Div` component assigned the ID 'my-output'.

**Code Snippet:**

```python
app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),

@callback(
    Output('my-output', 'children'),
    Input('my-input', 'value')
)
def update_output_div(input_value):
    return f'Output: {input_value}'
```

* **Explanation:**
    - Within the `app.layout` definition, the `id` property is used to assign unique identifiers to the components.
    - In the `@callback` decorator:
        - The `Output` argument specifies the component to be updated (referencing its ID 'my-output').
        - The `Input` argument indicates the component that triggers the callback update (referencing its ID 'my-input').

By adhering to this approach, we establish a clear connection between the components in the application layout and the corresponding inputs and outputs within the callback functions. This ensures that the callbacks interact with the intended components effectively.


**Alternative Approach: Direct Component Referencing in Callbacks**

While assigning unique IDs and referencing them in callbacks is a common practice, Dash offers an alternative approach:

* **Direct Component Usage:**
    - Components can be directly provided as inputs and outputs within callback functions, eliminating the need for explicit ID assignment.

* **Automatic ID Generation:**
    - When employing this method, Dash internally generates unique identifiers for the components.

* **Illustrative Example:**
    - The first example is presented again to demonstrate this approach.

**Code Snippet:**

```python
from dash import Dash, dcc, html, Input, Output, callback

app = Dash(__name__)

my_input = dcc.Input(value='initial value', type='text')
my_output = html.Div()

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        my_input
    ]),
    html.Br(),
    my_output
])


@callback(
    Output(my_output, 'children'),
    Input(my_input, 'value')
)
def update_output_div(input_value):
    return f'Output: {input_value}'


if __name__ == '__main__':
    app.run(debug=True)
```

* **Explanation:**
    - In this example, two components (`my_input` and `my_output`) are created and assigned to variables.
    - Within the `app.layout`, these variables are directly referenced, eliminating the need for separate ID assignments.
    - Similarly, the callback function directly references the component variables as arguments for `Output` and `Input`.

**Choosing the Right Approach:**

Both methods (using IDs or direct referencing) effectively establish the connection between components and callbacks. Selecting the most suitable approach often depends on personal preference and coding style.


**Utilizing the Walrus Operator in Python 3.8+ for Component Creation:**

* **Compatibility:** This approach is applicable in Python versions 3.8 and above.

* **Functionality:**
    - The walrus operator (`:=`) allows for combining variable assignment and expression evaluation within a single statement.

* **Example:**
    - The provided code snippet demonstrates the walrus operator in action for declaring component variables directly within the application layout.

**Code Snippet:**

```python
app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        my_input := dcc.Input(value='initial value', type='text')
    ]),
    html.Br(),
    my_output := html.Div(),
])

@callback(
    Output(my_output, 'children'),
    Input(my_input, 'value')
)
def update_output_div(input_value):
    return f'Output: {input_value}'
```

* **Explanation:**
    - In this example, the walrus operator is employed to create the `my_input` and `my_output` components while simultaneously assigning them to their respective variables within the layout definition.

**Comparison to Other Methods:**

This approach using the walrus operator offers an alternative to declaring variables separately and then referencing them in the layout. It can potentially enhance code conciseness, especially for brief component creations.

**Important Note:**

While the walrus operator provides a concise way to declare and assign component variables, it's crucial to remember that it's a relatively new language feature (introduced in Python 3.8). If compatibility with older Python versions is a concern, the traditional method of separate variable declaration might be preferred.


In [17]:
#run the example here using the walrus operator

## Recap of Dash Callbacks and App Functionality

* **Reactive UI Updates:**
    - Dash applications leverage a concept known as "reactive callbacks" to achieve dynamic UI updates.
    - These callbacks allow various UI elements' attributes or properties to be modified based on user interactions or other events within the application.

* **Customizable User Interfaces:**
    - This reactive approach empowers developers to create highly customizable user interfaces.
    - The application can respond and adapt to user input or changes in underlying data, providing a dynamic and interactive experience.

* **Component Properties:**
    - Callbacks can be used to modify a wide range of attributes associated with Dash components.
    - This encompasses virtually all properties of a component, enabling fine-grained control over the UI's appearance and behavior.

* **User Interaction:**
    - A subset of component properties, such as the `value` property of the `dcc.Dropdown` component, are specifically designed to be directly editable by users through their interaction with the application.


In essence, Dash applications combine user interactions with reactive callbacks to create user-driven, dynamic interfaces that can adapt and respond to user input and changes in the application's state.


# Interactive Visualizations Using Dash
Here is the paraphrased text in a neutral tone without omitting information:

* **Dash Core Components (dash.dcc):** This module provides components for building interactive dashboards in Python.
* **dcc.Graph component:** This component specifically renders visualizations using the plotly.js library.
* **Plotly.js:** An open-source JavaScript library that supports a wide variety of chart types (over 35) and offers high-quality outputs in both SVG and WebGL formats.
* **figure argument:** This argument within dcc.Graph aligns with the one used in plotly.py, allowing you to leverage the plotly.py documentation and resources for further exploration.
* **Attribute updates:** Similar to other Dash components, dcc.Graph can have its attributes modified through callback functions. It's important to note that only specific attributes respond to user interactions, such as those triggered by typing in a dcc.Input or selecting options in a dcc.Dropdown.
* **User-interactive attributes in dcc.Graph:** This component offers four properties that update based on user interaction:
    * hoverData: Captures information when the user hovers over data points.
    * clickData: Stores data related to clicked points.
    * selectedData: Reflects any selected regions within the graph.
    * relayoutData: Updates layout information based on user interactions.

Run this code to visualize and see the interactive attribute in `dcc.Graph`

```python
from dash import Dash, dcc, html, Input, Output, callback

import plotly.express as px

import json
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

df = pd.DataFrame({
    "x": [1,2,1,2],
    "y": [1,2,3,4],
    "customdata": [1,2,3,4],
    "fruit": ["apple", "apple", "orange", "orange"]
})

fig = px.scatter(df, x="x", y="y", color="fruit", custom_data=["customdata"])

fig.update_layout(clickmode='event+select')

fig.update_traces(marker_size=20)

app.layout = html.Div([
    dcc.Graph(
        id='basic-interactions',
        figure=fig
    ),

    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown("""
                **Hover Data**

                Mouse over values in the graph.
            """),
            html.Pre(id='hover-data', style=styles['pre'])
        ], className='three columns'),

        html.Div([
            dcc.Markdown("""
                **Click Data**

                Click on points in the graph.
            """),
            html.Pre(id='click-data', style=styles['pre']),
        ], className='three columns'),

        html.Div([
            dcc.Markdown("""
                **Selection Data**

                Choose the lasso or rectangle tool in the graph's menu
                bar and then select points in the graph.

                Note that if `layout.clickmode = 'event+select'`, selection data also
                accumulates (or un-accumulates) selected data if you hold down the shift
                button while clicking.
            """),
            html.Pre(id='selected-data', style=styles['pre']),
        ], className='three columns'),

        html.Div([
            dcc.Markdown("""
                **Zoom and Relayout Data**

                Click and drag on the graph to zoom or click on the zoom
                buttons in the graph's menu bar.
                Clicking on legend items will also fire
                this event.
            """),
            html.Pre(id='relayout-data', style=styles['pre']),
        ], className='three columns')
    ])
])


@callback(
    Output('hover-data', 'children'),
    Input('basic-interactions', 'hoverData'))
def display_hover_data(hoverData):
    return json.dumps(hoverData, indent=2)


@callback(
    Output('click-data', 'children'),
    Input('basic-interactions', 'clickData'))
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)


@callback(
    Output('selected-data', 'children'),
    Input('basic-interactions', 'selectedData'))
def display_selected_data(selectedData):
    return json.dumps(selectedData, indent=2)


@callback(
    Output('relayout-data', 'children'),
    Input('basic-interactions', 'relayoutData'))
def display_relayout_data(relayoutData):
    return json.dumps(relayoutData, indent=2)


if __name__ == '__main__':
    app.run(debug=True)
```

In [18]:
#run the code here

## Update Graphs on Hover
Let's update our world indicators example from the previous chapter by updating the time series when we hover over points in our scatter plot.

Run this code:

```python
from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_x_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_y_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run(debug=True)

```

In [19]:
# run the example there

Try moving the mouse over the points in the scatter plot on the left. Notice how the line graphs on the right update based on the point that you are hovering over.

**In class activity**

*Add this to the previous tasks for submission;*

*This is an individual activity*

What are the pros and cons of such type of interaction?

## Generic Crossfilter Recipe

For discussion, use this code:

```python
from dash import Dash, dcc, html, Input, Output, callback
import numpy as np
import pandas as pd
import plotly.express as px

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = Dash(__name__, external_stylesheets=external_stylesheets)

# make a sample data frame with 6 columns
np.random.seed(0)  # no-display
df = pd.DataFrame({"Col " + str(i + 1): np.random.rand(30) for i in range(6)})

app.layout = html.Div(
    [
        html.Div(
            dcc.Graph(id="g1", config={"displayModeBar": False}),
            className="four columns",
        ),
        html.Div(
            dcc.Graph(id="g2", config={"displayModeBar": False}),
            className="four columns",
        ),
        html.Div(
            dcc.Graph(id="g3", config={"displayModeBar": False}),
            className="four columns",
        ),
    ],
    className="row",
)


def get_figure(df, x_col, y_col, selectedpoints, selectedpoints_local):

    if selectedpoints_local and selectedpoints_local["range"]:
        ranges = selectedpoints_local["range"]
        selection_bounds = {
            "x0": ranges["x"][0],
            "x1": ranges["x"][1],
            "y0": ranges["y"][0],
            "y1": ranges["y"][1],
        }
    else:
        selection_bounds = {
            "x0": np.min(df[x_col]),
            "x1": np.max(df[x_col]),
            "y0": np.min(df[y_col]),
            "y1": np.max(df[y_col]),
        }

    # set which points are selected with the `selectedpoints` property
    # and style those points with the `selected` and `unselected`
    # attribute. see
    # https://medium.com/@plotlygraphs/notes-from-the-latest-plotly-js-release-b035a5b43e21
    # for an explanation
    fig = px.scatter(df, x=df[x_col], y=df[y_col], text=df.index)

    fig.update_traces(
        selectedpoints=selectedpoints,
        customdata=df.index,
        mode="markers+text",
        marker={"color": "rgba(0, 116, 217, 0.7)", "size": 20},
        unselected={
            "marker": {"opacity": 0.3},
            "textfont": {"color": "rgba(0, 0, 0, 0)"},
        },
    )

    fig.update_layout(
        margin={"l": 20, "r": 0, "b": 15, "t": 5},
        dragmode="select",
        hovermode=False,
        newselection_mode="gradual",
    )

    fig.add_shape(
        dict(
            {"type": "rect", "line": {"width": 1, "dash": "dot", "color": "darkgrey"}},
            **selection_bounds
        )
    )
    return fig


# this callback defines 3 figures
# as a function of the intersection of their 3 selections
@callback(
    Output("g1", "figure"),
    Output("g2", "figure"),
    Output("g3", "figure"),
    Input("g1", "selectedData"),
    Input("g2", "selectedData"),
    Input("g3", "selectedData"),
)
def callback(selection1, selection2, selection3):
    selectedpoints = df.index
    for selected_data in [selection1, selection2, selection3]:
        if selected_data and selected_data["points"]:
            selectedpoints = np.intersect1d(
                selectedpoints, [p["customdata"] for p in selected_data["points"]]
            )

    return [
        get_figure(df, "Col 1", "Col 2", selectedpoints, selection1),
        get_figure(df, "Col 3", "Col 4", selectedpoints, selection2),
        get_figure(df, "Col 5", "Col 6", selectedpoints, selection3),
    ]


if __name__ == "__main__":
    app.run(debug=True)
```

In [20]:
# Run the example here
#select 3 different regions and observe the changes in the visualization

* This example demonstrates cross-filtering functionality on a six-column dataset.
* Selecting data points in any scatter plot filters the entire dataset.
* Each selection triggers three graph callbacks, receiving the latest chosen regions from each plot.
* A pandas dataframe is then filtered based on the chosen points.
* The graphs are subsequently updated: selected points are highlighted, and the chosen region is visualized with a dashed rectangle.


In [21]:
from dash import Dash, dcc, html, Input, Output, callback

from numpy import pi
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app1 = Dash(__name__, external_stylesheets=external_stylesheets)

app1.layout = html.Div([
    dcc.Input(
        id='num-multi',
        type='number',
        value=5
    ),
    html.Table([
        html.Tr([html.Td(['2(x+3)']), html.Td(id='square')]),
        html.Tr([html.Td(['5x', html.Sup(1/2)]), html.Td(id='cube')]),
        html.Tr([html.Td(['pi(x', html.Sup('2'),')']), html.Td(id='twos')])
    ]),
])


@app1.callback(
    Output('square', 'children'),
    Output('cube', 'children'),
    Output('twos', 'children'),
    Input('num-multi', 'value'))
def callback_a(x):
    return 2*(x+3), 5*x**(1/2), pi*(x**2)


if __name__ == '__main__':
    app1.run(debug=True)

In [32]:
from dash import Dash, dcc, html, Input, Output, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app2 = Dash(__name__, external_stylesheets=external_stylesheets)

all_options = {
    'Cebu': ['Talisay', 'Naga', 'Carcar'],
    'Negros Occidental': ['Bais', 'Bayawan', 'Canlaon']
}
app2.layout = html.Div([
    dcc.RadioItems(
        list(all_options.keys()),
        'Cebu',
        id='provinces-radio',
    ),

    html.Hr(),

    dcc.RadioItems(id='cities-radio'),

    html.Hr(),

    html.Div(id='display-selected-values')
])


@app2.callback(
    Output('cities-radio', 'options'),
    Input('provinces-radio', 'value'))
def set_cities_options(selected_country):
    return [{'label': i, 'value': i} for i in all_options[selected_country]]


@app2.callback(
    Output('cities-radio', 'value'),
    Input('cities-radio', 'options'))
def set_cities_value(available_options):
    return available_options[0]['value']


@app2.callback(
    Output('display-selected-values', 'children'),
    Input('provinces-radio', 'value'),
    Input('cities-radio', 'value'))
def set_display_children(selected_country, selected_city):
    return f'{selected_city} is a city in {selected_country}'


if __name__ == '__main__':
    app2.run(debug=True)

In [28]:
from dash import Dash, dcc, html, Input, Output, State, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Input(id='input-1-state', type='text', value='Montréal'),
    dcc.Input(id='input-2-state', type='text', value='Canada'),
    html.Button(id='submit-button-state', n_clicks=0, children='Submit'),
    html.Div(id='output-state')
])


@callback(Output('output-state', 'children'),
              Input('submit-button-state', 'n_clicks'),
              State('input-1-state', 'value'),
              State('input-2-state', 'value'))
def update_output(n_clicks, input1, input2):
    return f'''
        The Button has been pressed {n_clicks} times,
        Input 1 is "{input1}",
        and Input 2 is "{input2}"
    '''


if __name__ == '__main__':
    app.run(debug=True)

In [31]:
from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_x_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_y_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run(debug=True)

The graphs area able to show the connection between multiple sets of data however the level of interactivity provided by hover data may be difficult to use.

Accidentally, moving the cursor over another element will cause the other graphs to change as well.

Additionally, we should not remain dependent solely on graph-cursor interactivity. A graph maybe able to show the desired elements (i.e. countries) in a tight cluster that will make searching for it difficult. A dropdown would have been preferred in such cases.

Ideally, this type of graph interactivity works best in small and aggregated datasets where items can be easily distinguished and selected.